# Credit Scorecard development case study

In [ ]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Data Loading

In [ ]:
data = pd.read_excel('data/Dataset-case.xlsx')

In [ ]:
# data.head()
data.sample(10)

In [ ]:
data.info()

In [ ]:
# Get the number of missing data points per column
data.isnull().sum()

**Missing Values**

| Column                  | Missing Values |
--------------------------|----------------|
| emp_length |               3342 |
| revol_util |                 33 |
| mort_acc |                 2039 |
| pub_rec_bankruptcies |       27 |

In [ ]:
# I should not remove any of the columns with missing values because they are all important
# Percedence of values missing is as follows:
data.isnull().mean().sort_values(ascending=False)

## Identify the target / dependent variable

In [ ]:
# loan_status is the target variable
# Where Default = 1 and Non-Default = 0
data['loan_status'].value_counts()

In [ ]:
# Drop loan_status2 column as it is not needed anymore
data.drop(columns = ['loan_status_2'], inplace = True)

In [ ]:
data.loan_status.value_counts(normalize=True)

In [ ]:
# Plot it as a bar chart
data.loan_status.value_counts(normalize=True).sort_values().plot(kind='bar')
plt.title('Loan Status')
# add Default instead of 1 and Non-Default instead of 0
plt.xticks([0, 1], ['Non-Default', 'Default'], rotation=0)
# show exact values
for i, v in enumerate(data.loan_status.value_counts(normalize=True).sort_values()):
    plt.text(i - 0.1, v + 0.01, str(round(v, 2)))
    
plt.show()

## Split the data into train and test sets

In [ ]:
# in every column remove spaces and replace with underscore
data.columns = data.columns.str.replace(' ', '')


In [ ]:
# Split the data into train and test (80-20 split)
# Use random_state = 42 to ensure that the results are reproducible
# Use stratify to ensure that the proportion of good and bad loans is the same in both train and test sets
X = data.drop(columns = 'loan_status', axis=1)
y = data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_test = X_train.copy(), X_test.copy()

In [ ]:
X_train.shape, X_test.shape

## Data Cleaning / Pre-processing

In [ ]:
data.info()

In [ ]:
# emp_length
# Fill in the missing values with 0.
def emp_length_converter(df, column):
    df[column].fillna(value = 0, inplace = True)

# Apply to X_train
emp_length_converter(X_train, 'emp_length')

X_train['emp_length'].unique()
X_train['emp_length'].value_counts()

In [ ]:
# issue_d

# Convert date columns to datetime format
# And create a new column as a difference between today and the respective date column
def date_converter(df, column):
    # store current month
    today_date = pd.to_datetime('2023-05-16')
    # convert to datetime format
    df[column] = pd.to_datetime(df[column], format = "%b-%y")
    # calculate the difference in months and add to a new column
    df['mths_since_' + column] = round(pd.to_numeric((today_date - df[column]) / np.timedelta64(1, 'M')))
    # make any resulting -ve values to be equal to the max date
    df['mths_since_' + column] = df['mths_since_' + column].apply(lambda x: df['mths_since_' + column].max() if x < 0 else x)
    # drop the original date column
    df.drop(columns = [column], inplace = True)

# apply to X_train
date_converter(X_train, 'issue_d')

# Check the new columns
print(X_train['mths_since_issue_d'].describe())



In [ ]:
# Plot the distribution of the new column
plt.figure(figsize = (10, 5))
sns.distplot(X_train['mths_since_issue_d'])
plt.title('Months Since Issue Date')
plt.show()


In [ ]:
# mort_acc column
X_train['mort_acc'].describe()

In [ ]:
# mort_acc - fill in the missing values with 0
# It is better replace the empty values with 0 instead of the mean because the mean is 1.68.
def mort_acc_converter(df, column):
    df[column].fillna(value = 0, inplace = True)

mort_acc_converter(X_train, 'mort_acc')

In [ ]:
# revol_util column - Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
X_train['revol_util'].describe()

In [ ]:
X_train['revol_util'].value_counts()

In [ ]:
# revol_util
# Fill in the missing values with the mean. Because the mean it is more representative than the median.
def revol_util_converter(df, column):
    df[column].fillna(value = df[column].mean(), inplace = True)

revol_util_converter(X_train, 'revol_util')

In [ ]:
# pub_rec_bankruptcies
X_train['pub_rec_bankruptcies'].value_counts()

In [ ]:
# pub_rec_bankruptcies column: Number of public record bankruptcies
def pub_rec_bankruptcies_converter(df, column):
    df[column].fillna(value = 0, inplace = True)

pub_rec_bankruptcies_converter(X_train, 'pub_rec_bankruptcies')

## Feature Selection

In [ ]:
# First divide training data into categorical and numerical subsets
X_train_num = X_train.select_dtypes(include = np.number).copy()
X_train_cat = X_train.select_dtypes(include = np.object).copy()

In [ ]:
# initialize an empty dictionary to store the results of chi-square test
chi_sq_results = {}

# loop over all the categorical variables
for col in X_train_cat.columns:
    chi, pi, dof, expected = chi2_contingency(pd.crosstab(y_train, X_train_cat[col]))

    chi_sq_results.setdefault('Feature', []).append(col)
    chi_sq_results.setdefault('p-value', []).append(round(pi, 10))


# convert the dictionary to a dataframe
chi_sq_results = pd.DataFrame(chi_sq_results)

# sort the dataframe by p_value
chi_sq_results.sort_values(by = 'p-value', ascending = True, ignore_index=True, inplace = True)

chi_sq_results


None of the categorical variable seem to have predictive power

## ANOVA F-Statistic for numerical variables

In [ ]:
X_train_num.info()

In [ ]:
# Since f_class_if does not accept missing values, wee will do a avery crude imputation of missing values
X_train_num.fillna(X_train_num.mean(), inplace = True)

# Calculate the F Statistic and corresponding p value
f_stat, p_value = f_classif(X_train_num, y_train)

# Create a dataframe to store the results
ANOVA_F_table = pd.DataFrame(data = {'Numerical_Feature': X_train_num.columns.values, 'F-Score': f_stat, 'p values': p_value.round(decimals=10)})
ANOVA_F_table.sort_values(by = ['F-Score'], ascending = False, ignore_index = True, inplace = True)
ANOVA_F_table

In [ ]:
# Calculate the pair-wise correlations between the variables
corrmat = X_train_num.corr()
plt.figure(figsize = (10, 10))
sns.heatmap(corrmat, annot = False, square = True, cmap = 'coolwarm');


- The ANOVA F-Statistic is used to identify the numerical variables that are most predictive of the target variable.
- Those columns are: 'pub_rec', 'emp_length', 'pub_rec_bankruptcies', 'open_acc', 'revol_bal'

In [ ]:
# drop_columns_list = ['pub_rec', 'emp_length', 'pub_rec_bankruptcies', 'open_acc', 'revol_bal']

# def col_to_drop(df, column_list):
#     df.drop(columns = column_list, inplace = True)

# # apply to X_train
# col_to_drop(X_train, drop_columns_list)

## Creaty dummy variables for categorical variables

In [ ]:
X_train_cat.columns

In [ ]:
# def create_dummy(df, column_list):
#     '''
#     This function will create dummy variables for the categorical variables

#     df: the dataframe
#     column_list: list of categorical columns
#     '''
#     df_dummies = []
#     for col in column_list:
#         df_dummies.append(pd.get_dummies(df[col], prefix = col, prefix_sep = ':'))
#     df_dummies = pd.concat(df_dummies, axis = 1)
#     df = pd.concat([df, df_dummies], axis = 1)
#     return df

# # apply to X_train
# X_train = create_dummy(X_train, ['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state'])


In [ ]:
X_train.sample(5)

## Update the test set

In [ ]:
# Apply the same transformations to the test set
emp_length_converter(X_test, 'emp_length')
date_converter(X_test, 'issue_d')
mort_acc_converter(X_test, 'mort_acc')
revol_util_converter(X_test, 'revol_util')
pub_rec_bankruptcies_converter(X_test, 'pub_rec_bankruptcies')
# col_to_drop(X_test, drop_columns_list) # <<< I don't think this is necessary!

# X_test = create_dummy(X_test, ['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state']) # <<< I don't think this is necessary!

# reindex the dummied test set variables to make sure all the feature columns in the train set are also available in the test set
X_test = X_test.reindex(labels=X_train.columns, axis=1, fill_value=0)

# Weight of Evidence (WoE) - Binning / Feature Engineering

- We will analyze both categorical and numerical features on their categorical/binned WoEs and IVs (Information Value) and then combine some of these binned categories together through a custom Python Class with fit_transform method.

In [ ]:
# Create copies of the 4 training sets to be preprocessed using WoE
X_train_prepr = X_train.copy()
y_train_prepr = y_train.copy()
X_test_prepr = X_test.copy()
y_test_prepr = y_test.copy()

## Analyze WoEs and IVs of discrete features

In [ ]:
# The function takes 3 arguments: a dataframe (X_train_prepr), a string (column name), and a dataframe (y_train_prepr).
# The function returns a dataframe as a result.
def woe_discrete(df, cat_variabe_name, y_df):
    df = pd.concat([df[cat_variabe_name], y_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df


sns.set()
# Function for plotting WoE across categoris that takes 2 arguments: a dataframe and a number
def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
    x = np.array(df_WoE.iloc[:, 0].apply(str))
    y = df_WoE['WoE']
    plt.figure(figsize=(18, 6))
    plt.plot(x, y, marker = 'o', linestyle = 'dotted', color = 'k')
    plt.xlabel(df_WoE.columns[0])
    plt.ylabel('Weight of Evidence')
    plt.title(str('Weight of Evidence by ' + df_WoE.columns[0]))
    plt.xticks(rotation = rotation_of_x_axis_labels)    


### 'sub_grade'

In [ ]:
df = woe_discrete(X_train_prepr, 'sub_grade', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

In [ ]:
# Combine sub_grades:
#  A1,A2,A3,A4,A5 as A
#  B1,B2,B3,B4,B5 as B etc
# Create a dictionary to map the sub_grades to the grades
sub_grade_dict = {'A1':'A', 'A2':'A', 'A3':'A', 'A4':'A', 'A5':'A',
                    'B1':'B', 'B2':'B', 'B3':'B', 'B4':'B', 'B5':'B',
                    'C1':'C', 'C2':'C', 'C3':'C', 'C4':'C', 'C5':'C',
                    'D1':'D', 'D2':'D', 'D3':'D', 'D4':'D', 'D5':'D',
                    'E1':'E', 'E2':'E', 'E3':'E', 'E4':'E', 'E5':'E',
                    'F1':'F', 'F2':'F', 'F3':'F', 'F4':'F', 'F5':'F',
                    'G1':'G', 'G2':'G', 'G3':'G', 'G4':'G', 'G5':'G'}


# Apply the dictionary to the sub_grade column
X_train_prepr['sub_grade'] = X_train_prepr['sub_grade'].map(sub_grade_dict)
X_test_prepr['sub_grade'] = X_test_prepr['sub_grade'].map(sub_grade_dict)

In [ ]:
# Plot the WoE for the new sub_grade column
df = woe_discrete(X_train_prepr, 'sub_grade', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

We can see from the above graph that there is a continuous increase in WoE across the different grades. Therefore, we do not need to combine any features together and should leave all these 7 grades as they are

### home_ownership

In [ ]:
df = woe_discrete(X_train_prepr, 'home_ownership', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

### verification_status 

In [ ]:
df = woe_discrete(X_train_prepr, 'verification_status', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

### purpose

In [ ]:
df = woe_discrete(X_train_prepr, 'purpose', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

### addr_state

In [ ]:
df = woe_discrete(X_train_prepr, 'addr_state', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

### Analyze WoEs and IVs of numeric features

In [ ]:
# We define a function to calculate WoE of continuous variables. This is same as the function we defined earlier for discrete variables.
# The only difference are the 2 commented lines of code in the function that results in the df being sorted by continuous variable values
def woe_ordered_continuous(df, continuous_variabe_name, y_df):
    df = pd.concat([df[continuous_variabe_name], y_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    #df = df.sort_values(['WoE'])
    #df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [ ]:
X_train_prepr['loan_amnt_factor'] = pd.cut(X_train_prepr['loan_amnt'], 10)
df = woe_ordered_continuous(X_train_prepr, 'loan_amnt_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

In [ ]:
df = woe_ordered_continuous(X_train_prepr, 'term', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

In [ ]:
X_train_prepr['int_rate_factor'] = pd.cut(X_train_prepr['int_rate'], 10)

df = woe_ordered_continuous(X_train_prepr, 'int_rate_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

In [ ]:
X_train_prepr['installment_factor'] = pd.cut(X_train_prepr['installment'], 10)

df = woe_ordered_continuous(X_train_prepr, 'installment_factor', y_train_prepr)
df


In [ ]:
plot_by_woe(df)

In [ ]:
X_train_prepr['annual_inc_factor'] = pd.cut(X_train_prepr['annual_inc'], 20)
# 
df = woe_ordered_continuous(X_train_prepr, 'annual_inc_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

In [ ]:
# Most of them are < 350.000
X_train_prepr_temp = X_train_prepr[X_train_prepr['annual_inc'] <= 350000].copy()

X_train_prepr_temp['annual_inc_factor'] = pd.cut(X_train_prepr_temp['annual_inc'], 20)
# 
df = woe_ordered_continuous(X_train_prepr_temp, 'annual_inc_factor', y_train_prepr[X_train_prepr_temp.index])
df


In [ ]:
plot_by_woe(df, 90)

In [ ]:
# dti
X_train_prepr['dti_factor'] = pd.cut(X_train_prepr['dti'], 20)
df = woe_ordered_continuous(X_train_prepr, 'dti_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

In [ ]:
# fico_range_low and fico_range_high
X_train_prepr['fico_range_low_factor'] = pd.cut(X_train_prepr['fico_range_low'], 20)
df = woe_ordered_continuous(X_train_prepr, 'fico_range_low_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

In [ ]:
# fico_range_low and fico_range_high
X_train_prepr['fico_range_high_factor'] = pd.cut(X_train_prepr['fico_range_high'], 20)
df = woe_ordered_continuous(X_train_prepr, 'fico_range_high_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

In [ ]:
X_train_prepr['mort_acc_factor'] = pd.cut(X_train_prepr['mort_acc'], 10)
df = woe_ordered_continuous(X_train_prepr, 'mort_acc_factor', y_train_prepr)
df


In [ ]:
plot_by_woe(df, 90)

In [ ]:
# X_train_prepr['age'].describe()

In [ ]:
X_train_prepr['age_factor'] = pd.cut(X_train_prepr['age'], 4)
df = woe_ordered_continuous(X_train_prepr, 'age_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df, 90)

In [ ]:
X_train_prepr['pay_status_factor'] = pd.cut(X_train_prepr['pay_status'], 10)
df = woe_ordered_continuous(X_train_prepr, 'pay_status_factor', y_train_prepr)
df

In [ ]:
plot_by_woe(df)

# Define Custom Class from WoE Binning and Feature Engineering

In [ ]:
ref_categories = ['home_ownership:OTHER',  'verification_status:Not Verified',  'purpose:home_other_debt_moving_medical',  'loan_amnt:>32201', 'term:60', 'int_rate:>25.855', 'installment:>1071.177', 'annual_inc:>298505', 'dti:>39.9522', 'fico_range_low:>771.0', 'fico_range_high:>738.4', 'age:>60', 'pay_status:>0.2']


In [ ]:
import numpy as np
from numpy import ndarray
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin



class WoE_Binning(BaseEstimator, TransformerMixin):
    def __init__(self, X):
        self.X = X
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # sub_grade
        # X_new = X.loc[:, 'sub_grade:A':'sub_grade:G']
        # Define X_new to be the same as X
        X_new = X.copy()
        
        # home ownership
        X_new['home_ownership:OWN'] = X.loc[:,'home_ownership:OWN']
        X_new['home_ownership:RENT'] = X.loc[:,'home_ownership:RENT']
        X_new['home_ownership:OTHER'] = X.loc[:,'home_ownership:OTHER']
        X_new['home_ownership:NONE'] = X.loc[:,'home_ownership:NONE']
        X_new['home_ownership:ANY_MORTGAGE'] = sum([X['home_ownership:ANY'], X['home_ownership:MORTGAGE']])

        # verification_status
        X_new = pd.concat([X_new, X.loc[:, 'verification_status:Not Verified':'verification_status:Verified']], axis=1)
        
        # purpose
        X_new['purpose:small_business'] = X.loc[:, 'purpose:small_business']
        ## Let's combine features with low WoE
        X_new['purpose:home_other_debt_moving_medical'] = sum([X['purpose:home_improvement'], X['purpose:other'], X['purpose:debt_consolidation'], X['purpose:moving']])
        
        # addr_state - inf IV let's not consider it

        # loan_amnt
        X_new['loan_amnt:<12701'] = np.where((X['loan_amnt'] <= 12701), 1, 0)
        X_new['loan_amnt:12701-24401'] = np.where((X['loan_amnt'] > 12701) & (X['loan_amnt'] <= 24401), 1, 0)
        X_new['loan_amnt:24401-32201'] = np.where((X['loan_amnt'] > 24401) & (X['loan_amnt'] <= 32201), 1, 0)
        X_new['loan_amnt:>32201'] = np.where((X['loan_amnt'] > 32201), 1, 0)

        # term
        X_new['term:36'] = np.where((X['term'] == 36), 1, 0)
        X_new['term:60'] = np.where((X['term'] == 60), 1, 0)

        # int_rate
        X_new['int_rate:<13.015'] = np.where((X['int_rate'] <= 13.015), 1, 0)
        X_new['int_rate:13.015-20.719'] = np.where((X['int_rate'] > 13.015) & (X['int_rate'] <= 20.719), 1, 0)
        X_new['int_rate:20.719-25.855'] = np.where((X['int_rate'] > 20.719) & (X['int_rate'] <= 25.855), 1, 0)
        X_new['int_rate:>25.855'] = np.where((X['int_rate'] > 25.855), 1, 0)

        # installment
        X_new['installment:<327.987'] = np.where((X['installment'] <= 327.987), 1, 0)
        X_new['installment:327.987-1071.177'] = np.where((X['installment'] > 327.987) & (X['installment'] <= 1071.177), 1, 0)
        X_new['installment:>1071.177'] = np.where((X['installment'] > 1071.177), 1, 0)

        # annual_inc
        X_new['annual_inc:missing'] = np.where(X['annual_inc'].isnull(), 1, 0)
        X_new['annual_inc:<75357'] = np.where((X['annual_inc'] <= 75357), 1, 0)
        X_new['annual_inc:75357-161183'] = np.where((X['annual_inc'] > 75357) & (X['annual_inc'] <= 161183), 1, 0)
        X_new['annual_inc:161183-195513'] = np.where((X['annual_inc'] > 161183) & (X['annual_inc'] <= 195513), 1, 0)
        X_new['annual_inc:195513-247009]'] = np.where((X['annual_inc'] > 195513) & (X['annual_inc'] <= 247009), 1, 0)
        X_new['annual_inc:247009-264174]]'] = np.where((X['annual_inc'] > 247009) & (X['annual_inc'] <= 264174), 1, 0)
        X_new['annual_inc:264174-281339]]'] = np.where((X['annual_inc'] > 264174) & (X['annual_inc'] <= 281339), 1, 0)
        X_new['annual_inc:281339-298505]]'] = np.where((X['annual_inc'] > 281339) & (X['annual_inc'] <= 298505), 1, 0)
        X_new['annual_inc:>298505]]'] = np.where((X['annual_inc'] > 298505), 1, 0)

        # dti
        X_new['dti:<19.977'] = np.where((X['dti'] <= 19.977), 1, 0)
        X_new['dti:19.977-37.455'] = np.where((X['dti'] > 19.977) & (X['dti'] <= 37.455), 1, 0)
        X_new['dti:37.455-39.9522'] = np.where((X['dti'] > 37.455) & (X['dti'] <= 39.9522), 1, 0)
        X_new['dti:>39.9522'] = np.where((X['dti'] > 39.9522), 1, 0)

        # fico_range_low
        X_new['fico_range_low:<697.01'] = np.where((X['fico_range_low'] <= 697.01), 1, 0)
        X_new['fico_range_low:697.01-771.0'] = np.where((X['fico_range_low'] > 697.01) & (X['fico_range_low'] <= 771.0), 1, 0)
        X_new['fico_range_low:>771.0'] = np.where((X['fico_range_low'] > 771.0), 1, 0)

        # fico_range_high
        X_new['fico_range_high:<691.9'] = np.where((X['fico_range_high'] <= 691.9), 1, 0)
        X_new['fico_range_high:691.9-738.4'] = np.where((X['fico_range_high'] > 691.9) & (X['fico_range_high'] <= 738.4), 1, 0)
        X_new['fico_range_high:>738.4'] = np.where((X['fico_range_high'] > 738.4), 1, 0)

        # mort_acc - do not consider it

        # age
        X_new['age:<30'] = np.where((X['age'] <= 30), 1, 0)
        X_new['age:30-60'] = np.where((X['age'] > 30) & (X['age'] <= 60), 1, 0)
        X_new['age:>60'] = np.where((X['age'] > 60) , 1, 0)
        
        # pay_status
        X_new['pay_status:<-0.9'] = np.where((X['pay_status'] <= -0.9), 1, 0)
        X_new['pay_status:-0.9-0.2'] = np.where((X['pay_status'] > -0.9) & (X['pay_status'] <= 0.2), 1, 0)
        X_new['pay_status:>0.2'] = np.where((X['pay_status'] > 0.2), 1, 0)
        return X_new
    


# PD Model Estimation

In [ ]:
# reconfirm shape of the 4 datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# define modeling pipeline
reg = LogisticRegression(max_iter=1000, class_weight = 'balanced')
woe_transform = WoE_Binning(X)
pipeline = Pipeline(steps=[('woe', woe_transform), ('model', reg)])

# define cross-validation criteria. RepeatedStratifiedKFold automatially takes care of the class imbalance while splitting
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# fit and evaluate the logistic regression pipeline with cross-validation as defined in cv
scores = cross_val_score(pipeline, X_train, y_train, scoring = 'roc_auc', cv = cv)
AUROC = np.mean(scores)
GINI = AUROC * 2 - 1

# print the mean AUROC score and Gini
print('Mean AUROC: %.4f' % (AUROC))
print('Gini: %.4f' % (GINI))

In [ ]:
# X.info()
X.head()

In [ ]:
# pretty impressive scores for the first time, now let's fit the pipeline on the whole training set
pipeline.fit(X_train, y_train)

In [ ]:
# first create a transformed training set through our WoE_Binning custom class
X_train_woe_transformed = woe_transform.fit_transform(X_train)
# Store the column names in X_train as a list
feature_name = X_train_woe_transformed.columns.values
# Create a summary table of our logistic regression model
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
# Create a new column in the dataframe, called 'Coefficients', with row values the transposed coefficients from the 'LogisticRegression' model
summary_table['Coefficients'] = np.transpose(pipeline['model'].coef_)
# Increase the index of every row of the dataframe with 1 to store our model intercept in 1st row
summary_table.index = summary_table.index + 1
# Assign our model intercept to this new row
summary_table.loc[0] = ['Intercept', pipeline['model'].intercept_[0]]
# Sort the dataframe by index
summary_table.sort_index(inplace = True)
summary_table

# Predictions

In [ ]:
# make preditions on our test set
y_hat_test = pipeline.predict(X_test)
# get the predicted probabilities
y_hat_test_proba = pipeline.predict_proba(X_test)
# select the probabilities of only the positive class (class 1 - default) 
y_hat_test_proba = y_hat_test_proba[:][: , 1]

In [ ]:
# we will now create a new DF with actual classes and the predicted probabilities
# create a temp y_test DF to reset its index to allow proper concaternation with y_hat_test_proba
y_test_temp = y_test.copy()
y_test_temp.reset_index(drop = True, inplace = True)
y_test_proba = pd.concat([y_test_temp, pd.DataFrame(y_hat_test_proba)], axis = 1)
# check the shape to make sure the number of rows is same as that in y_test
y_test_proba.shape

In [ ]:
# Rename the columns
y_test_proba.columns = ['y_test_class_actual', 'y_hat_test_proba']
# Makes the index of one dataframe equal to the index of another dataframe.
y_test_proba.index = X_test.index
y_test_proba.head()

## Confusion Matrix and AUROC on Test Set

In [ ]:
# assign a threshold value to differentiate good with bad
tr = 0.5
# crate a new column for the predicted class based on predicted probabilities and threshold
# We will determine this optimat threshold later in this project
y_test_proba['y_test_class_predicted'] = np.where(y_test_proba['y_hat_test_proba'] > tr, 1, 0)
# create the confusion matrix
confusion_matrix(y_test_proba['y_test_class_actual'], y_test_proba['y_test_class_predicted'], normalize = 'all')

In [ ]:
# get the values required to plot a ROC curve
fpr, tpr, thresholds = roc_curve(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
# plot the ROC curve
plt.plot(fpr, tpr)
# plot a secondary diagonal line, with dashed line style and black color to represent a no-skill classifier
plt.plot(fpr, fpr, linestyle = '--', color = 'k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve');

In [ ]:
# Calculate the Area Under the Receiver Operating Characteristic Curve (AUROC) on our test set
AUROC = roc_auc_score(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
AUROC

In [ ]:
# calculate Gini from AUROC
Gini = AUROC * 2 - 1
Gini

In [ ]:
# draw a PR curve
# calculate the no skill line as the proportion of the positive class
no_skill = len(y_test[y_test == 1]) / len(y)
# plot the no skill precision-recall curve
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')

# calculate inputs for the PR curve
precision, recall, thresholds = precision_recall_curve(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
# plot PR curve
plt.plot(recall, precision, marker='.', label='Logistic')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('PR curve');

In [ ]:
# calculate PR AUC
auc_pr = auc(recall, precision)
auc_pr

# Applying the Model - Scorecard Creation

In [ ]:
summary_table

In [ ]:
# We create a new dataframe with one column. Its values are the values from the 'reference_categories' list. We name it 'Feature name'.
df_ref_categories = pd.DataFrame(ref_categories, columns = ['Feature name'])
# We create a second column, called 'Coefficients', which contains only 0 values.
df_ref_categories['Coefficients'] = 0
df_ref_categories

In [ ]:
# Concatenates two dataframes.
df_scorecard = pd.concat([summary_table, df_ref_categories])
# We reset the index of a dataframe.
df_scorecard.reset_index(inplace = True)
df_scorecard

In [ ]:
# create a new column, called 'Original feature name', which contains the value of the 'Feature name' column, up to the column symbol.
df_scorecard['Original feature name'] = df_scorecard['Feature name'].str.split(':').str[0]
df_scorecard

In [ ]:
# Define the min and max threshholds for our scorecard
min_score = 300
max_score = 850

In [ ]:
# calculate the sum of the minimum coefficients of each category within the original feature name
min_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
# calculate the sum of the maximum coefficients of each category within the original feature name
max_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()
# create a new columns that has the imputed calculated Score based on the multiplication of the coefficient by the ratio of the differences between
# maximum & minimum score and maximum & minimum sum of cefficients.
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score) / (max_sum_coef - min_sum_coef)
# update the calculated score of the Intercept (i.e. the default score for each loan)
df_scorecard.loc[0, 'Score - Calculation'] = ((df_scorecard.loc[0,'Coefficients'] - min_sum_coef) / (max_sum_coef - min_sum_coef)) * (max_score - min_score) + min_score
# round the values of the 'Score - Calculation' column and store them in a new column
df_scorecard['Score - Preliminary'] = df_scorecard['Score - Calculation'].round()
df_scorecard

In [ ]:
# check the min and max possible scores of our scorecard
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].min().sum()
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].max().sum()
print(min_sum_score_prel)
print(max_sum_score_prel)

In [ ]:
# so both our min and max scores are out by +1. we need to manually adjust this
# Which one? We'll evaluate based on the rounding differences of the minimum category within each Original Feature Name.
pd.options.display.max_rows = 102
df_scorecard['Difference'] = df_scorecard['Score - Preliminary'] - df_scorecard['Score - Calculation']
df_scorecard

In [ ]:
# look like we can get by deducting 1 from the Intercept
df_scorecard['Score - Final'] = df_scorecard['Score - Preliminary']
df_scorecard.loc[0, 'Score - Final'] = 598
df_scorecard

In [ ]:
# Recheck min and max possible scores
print(df_scorecard.groupby('Original feature name')['Score - Final'].min().sum())
print(df_scorecard.groupby('Original feature name')['Score - Final'].max().sum())

In [ ]:
## Calculating credit scores for all observations in the test data set

In [ ]:
# first create a transformed test set through our WoE_Binning custom class
X_test_woe_transformed = woe_transform.fit_transform(X_test)
# insert an Intercept column in its beginning to align with the # of rows in scorecard
X_test_woe_transformed.insert(0, 'Intercept', 1)
X_test_woe_transformed.head()

In [ ]:
# get the list of our final scorecard scores
scorecard_scores = df_scorecard['Score - Final']
# check the shapes of test set and scorecard before doing matrix dot multiplication
print(X_test_woe_transformed.shape)
print(scorecard_scores.shape)

In [ ]:
# we can see that the test set has 17 less columns than the rows in scorecard due to the reference categories
# since the reference categories will always be scored as 0 based on the scorecard, it is safe to add these categories to the end of test set with 0 values
X_test_woe_transformed = pd.concat([X_test_woe_transformed, pd.DataFrame(dict.fromkeys(ref_categories, [0] * len(X_test_woe_transformed)), 
                                                                         index = X_test_woe_transformed.index)], axis = 1)
# Need to reshape scorecard_scores so that it is (102,1) to allow for matrix dot multiplication
scorecard_scores = scorecard_scores.values.reshape(102, 1)
print(X_test_woe_transformed.shape)
print(scorecard_scores.shape)

In [ ]:
# matrix dot multiplication of test set with scorecard scores
y_scores = X_test_woe_transformed.dot(scorecard_scores)
y_scores.head()